In [34]:
import sys
import tkinter
from selenium import webdriver
from tkinter import messagebox
import urllib
import time
import re,codecs
import pandas as pd
from selenium.common.exceptions import NoSuchElementException

In [35]:
from keras.models import load_model
model = load_model('./top_layers_v2.iv3.hdf5')

In [36]:
root = tkinter.Tk()
root.withdraw()

capabilities = {
    'browserName': 'chrome',
    'chrome.detach': True,
    'chromeOptions': {
        'useAutomationExtension': False,
        'forceDevToolsScreenshot': True,
        'args': ['--start-maximized', '--disable-infobars']
    }
}
# driver = webdriver.Chrome(desired_capabilities=capabilities)
options = webdriver.ChromeOptions()
options.add_experimental_option("useAutomationExtension", False)
driver = webdriver.Chrome(r"C:\CHROMEDRIVER\chromedriver", desired_capabilities=capabilities)
driver2 = webdriver.Chrome(r"C:\CHROMEDRIVER\chromedriver", desired_capabilities=capabilities)

# accounts = ['voyaged','bruisedpassports','nationalparkservice','nxplore','doyoutravel','michiwohlleben','heykelseyj','Funlifecrisis','jannikobenhoff','chrisburkard','theglobewanderer','erubes1','alexstrohl','nateinthewild','scottslawinski','visualsofjulius','ilhan1077','niels.meiser','Nortrip','mittvestland','nevinixavier','theolator','Helpwphoto','mrdanielhan','dr.iyasar','ninjarod','Kayvanhuesseling','jordhammond']
iceland_accounts = ['frederikopdeweegh','iceland','aurorareykjavik','lost.in.iceland','stunningicelandd','eyeofshe','beyondthelands','Josefwittibschlager','icelandtravel','patheight','withluke','erychist','the.newborn','arnarkristjans_photography','hiddeniceland','unlimitediceland','katcraats','mbeiter','meeeeeeexx','your_nordic','inspiredbyiceland','dreamchasersiceland','Icelandic_explorer','arvids']
# iceland_accounts = ['mbeiter','arvids','icelandtravel']

In [37]:
driver.get("https://www.instagram.com/accounts/login/")
driver2.get("https://www.instagram.com/accounts/login/")
time.sleep(2)

In [38]:
username = driver.find_element_by_xpath('//*[@id="react-root"]/section/main/div/article/div/div[1]/div/form/div[2]/div/label/input')
password = driver.find_element_by_xpath('//*[@id="react-root"]/section/main/div/article/div/div[1]/div/form/div[3]/div/label/input')
username.send_keys("choudharyparidhi")
password.send_keys("Tooshort2bsad!")
driver.find_element_by_xpath('//*[@id="react-root"]/section/main/div/article/div/div[1]/div/form/div[4]/button').click()
username = driver2.find_element_by_xpath('//*[@id="react-root"]/section/main/div/article/div/div[1]/div/form/div[2]/div/label/input')
password = driver2.find_element_by_xpath('//*[@id="react-root"]/section/main/div/article/div/div[1]/div/form/div[3]/div/label/input')
username.send_keys("")
password.send_keys("")
driver2.find_element_by_xpath('//*[@id="react-root"]/section/main/div/article/div/div[1]/div/form/div[4]/button').click()
time.sleep(2)

In [39]:
save_address = "./app_iceland"

In [40]:
from bs4 import BeautifulSoup
def getLatLong(driver,link):
    fp = driver.get(link)
    source = driver.page_source
    soup = BeautifulSoup(source)
    latitude_tag = soup.find("meta",  property="place:location:latitude")
    longitude_tag = soup.find("meta",  property="place:location:longitude")
    latitude = latitude_tag["content"] if latitude_tag else None
    longitude = longitude_tag["content"] if longitude_tag else None
    return latitude, longitude

In [41]:
latest_dates = pd.read_csv("artists_latest_post.csv")

In [42]:
latest_dates['latest_post_date'] = latest_dates['latest_post_date'].apply(lambda x: x[:4]+"-"+x[5:7]+"-"+x[8:10]+"T"+x[11:13]+":"+x[14:16]+"."+x[17:19]+"Z")

In [2]:
import scipy as sp
import PIL
import scipy.ndimage as spi
import matplotlib.pyplot as plt
import glob
import numpy as np
import pandas as pd
import h5py
import os
from math import ceil, sqrt
import tensorflow as tf
from imageio import imread
from PIL import Image
from keras.models import load_model
from keras import backend as K
test_images = []
data_info = []
# f_read = codecs.open("data_app_iceland.txt","w",encoding='utf-8-sig')
for account_name in iceland_accounts:
    try:
        if account_name in list(latest_dates['artist']):
            latest_post_by_artist = list(latest_dates[latest_dates['artist']==account_name]['latest_post_date'])[0]
            link_add = "https://www.instagram.com/"+account_name+"/"
            hrefs = []
            driver.get(link_add)
            time.sleep(1)
            num_posts = (driver.find_element_by_xpath('//*[@id="react-root"]/section/main/div/header/section/ul/li[1]/span/span').text)
            num_posts = int(num_posts.replace(',',''))
            max_num_posts = min(num_posts,10)
            elems = driver.find_elements_by_css_selector(".v1Nh3.kIKUG._bz0w")
            for x in elems:
                hrefs.append(x.find_element_by_xpath(".//a").get_attribute("href"))
            i=0
            while len(hrefs)<max_num_posts:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(1)
                if i==0:
                    start_index = 12
                elif i==1:
                    start_index = 24
                else:
                    start_index = 30
                elems = driver.find_elements_by_css_selector(".v1Nh3.kIKUG._bz0w")
                for x in elems[start_index:]:
                    hrefs.append(x.find_element_by_xpath(".//a").get_attribute("href"))    
            for href in hrefs:
                try:
                    driver.get(href)
                    current_img_time = driver.find_elements_by_tag_name("time")[0].get_attribute("datetime")
                    print(current_img_time)
                    if current_img_time<=latest_post_by_artist:
                        break
                    else:
                        id_ = account_name+"_"+current_img_time.replace("-","_").replace(":","_").replace("T","_")
                        try:
                            location_link = (driver.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div/article/header/div[2]/div[2]/div[2]/a').get_attribute("href"))
                            latitude, longitude = getLatLong(driver2,location_link)
                        except NoSuchElementException as e:
                            latitude, longitude = [None,None]
                            pass
                        datetime = current_img_time
                        images = (driver.find_elements_by_tag_name("img"))
                        link_image = (images[1].get_attribute("srcset").split(",")[-1].split(" ")[0])
                        driver.get(link_image)
                        img = driver.find_element_by_tag_name("img")
                        src = img.get_attribute('src')
                        filename = save_address+"/"+id_+".png"
                        urllib.request.urlretrieve(src,filename)
                        im1 = Image.open(filename)
                        im2 = im1.resize((299, 299), Image.NEAREST)
                        test_image = np.array(im2)
                        test_images.append(test_image)
                        data_info.append([id_,account_name,datetime,str(latitude),str(longitude),filename])
                except Exception as e:
                    print(str(e))
                    continue
    except Exception as e:
        print(e)
        continue
test_images = np.array(test_images)
data_info = np.array(data_info)
f_read.close()

In [47]:
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from sklearn.preprocessing import OneHotEncoder
%matplotlib inline
np.random.seed(42)
print(test_images.shape)
enc = OneHotEncoder(sparse=False)
categories = np.array(['geyser','glacier','road','northern lights','sea','mountain','people','waterfall']).reshape(8,1)
enc.fit_transform(categories)
predictions_enc = model.predict(test_images,verbose=1)
predictions_unenc = enc.inverse_transform(predictions_enc)

(234, 299, 299, 3)
234/234 [==============================] - 214s 913ms/step


In [57]:
def calculateDistance(i1, i2):
    return np.sqrt(np.sum((i1-i2)**2))

In [58]:
calculateDistance(test_images[0],match_with_images[0])

5124.807313450917

In [1]:
distances = np.array(distances)
city_state_location = np.array(city_state_location)
match_with_images = np.array(match_with_images)
np.column_stack((match_with_images,city_state_location))

In [70]:
match_with_images = []
city_state_location = []
import codecs
with codecs.open("iceland_mountain_lat_long_city_state.txt",'r',encoding="utf-8-sig") as f:
    for line in f:
        elems = line.strip("r").split("|")
        storage_location,latitude,longitude,city,state = elems
        if ((latitude is None or longitude is None) or (latitude=='64.830555555556' and longitude=='-17.986666666667')):
            continue
        else:
            im1 = Image.open(storage_location)
            im2 = im1.resize((299, 299), Image.NEAREST)
            test_image = np.array(im2)
            match_with_images.append(test_image)
            city_state_location.append([latitude,longitude,city,state])